In [2]:
import requests
from requests import get
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time
import os
from urllib.request import urlopen
import urllib.request, urllib.error, urllib.parse
import mysql.connector
import sqlalchemy
import json

# Step 1

Write code that saves allnon-sponsoreditem URLs (the URL you go to when clicking on an item) to the file "deals.txt" in the same directory as your code. 

In [4]:
#All the items are non sponsored - so scrapping all of them from first 10 pages
#Creating file - deals.txt with the item URLs


url = 'https://www.ebay.com/e/daily-deals/hiw-presidents-day-deals-white-sale?'

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36",
"Upgrade-Insecure-Requests":"1"}

count=0

file1 = open("deals.txt","w") 

for i in range(1,11):
    url1=url+'_pgn='+str(i)
    page = requests.get(url1, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    divs = soup.find_all("div",{"class":"s-item__info clearfix"})
    for div in divs:
        for tags in div.find_all("a"):
            item_link = tags.get('href')
            x=bool(re.search("^.*itm",item_link))
            if x == True:
                count=count+1
                file1.write(item_link+"\n") 
    time.sleep(10)     
            
file1.close() 

In [5]:
print(count) #total 219 item URLs

#There are only 8 pages for items but as instructed scrapping 10 pages which might result in few identical results

216


# Step 2

Write a program that opens the above file and downloads each of the pages (URLs) into the folders "deals". Each file should be named as "<item-id>.html" where you replace "item-id" with the ID of the item you are saving. E.g., "264616053293.html" for the item with ID "264616053293".

In [9]:
#Opening the deals.txt file

file1 = open("deals.txt","r+") 

In [25]:
#Downloading item pages

path = '/Users/shreyashah/Desktop/UC Davis/Courses/Winter Quarter/Data Design and Representation/Homeworks/Midterm/deals'
os.chdir(path)

#Path needs to be changed as this is local to my computer

count1=0  
    
while True:
    try:
        count1 = count1 + 1 
   
        # Get next line from file
        line = file1.readline()
       # print(line)

        # if line is empty
        # end of file is reached
        if not line:
            break

        response = urllib.request.urlopen(str(line))
        webContent = response.read()
        text = re.findall("/[0-9]{12}?", str(line))
        f_name = text[0][1:]
        f = open(f_name+'.html', 'wb')

        f.write(webContent)
        f.close
    except:
        pass
    time.sleep(5)
    

In [28]:
file1.close()

# Step 3

Write a separate piece of code that loops through the pages you downloaded in step 2 and opens and parses them into a Python or Java xxxxsoup-object. Identify and select:seller name, seller score, item price, item price, list price, # items sold, title, returns allowed(true / false), shipping price, condition (e.g., used, new, like new, seller refurbished, ...).

In [10]:
#Selecting and storing variables of interest - seller name, seller score, item price, item price, list price, # items sold, title, returns allowed (true / false), shipping price, condition


path = '/Users/shreyashah/Desktop/UC Davis/Courses/Winter Quarter/Data Design and Representation/Homeworks/Midterm'
os.chdir(path)
file1 = open("deals.txt","r+") 

item_id_list = []

while True:
  
    # Get next line from file
    line = file1.readline()
    
    # if line is empty
    # end of file is reached
    if not line:
        break

    text = re.findall("/[0-9]{12}?", str(line))
    item_id_list.append(text[0][1:]) #list of 219 items ids

In [15]:
#Saving info for all 219 items 

path = '/Users/shreyashah/Desktop/UC Davis/Courses/Winter Quarter/Data Design and Representation/Homeworks/Midterm/deals'
os.chdir(path)

column_names = ["Seller Name", "Seller Score", "Item Price", "List Price", "Number of Items Sold", "Returns Allowed", "Shipping Price", "Condition"]
df = pd.DataFrame(columns = column_names) #New dataframe to save the values of all the variables

for i in range(0,2): 

    f = open(item_id_list[i]+'.html')
    time.sleep(10)
    
    soup = BeautifulSoup(f, 'html.parser')
    
    #Seller Name
    divs1 = soup.find_all("span",{"class":"mbg-nw"})
    for div in divs1:
        df.loc[i,"Seller Name"] = div.text
        
        
    #Seller Score
    divs2 = soup.find_all("div",{"id":"si-fb"})
    for div in divs2:
        seller_score = re.findall("[0-9].+%", div.text)
        df.loc[i,"Seller Score"] = seller_score[0][:-1]
    
        
    #Item Price
    time.sleep(10)
    divs3 = soup.find("span",{"class":"notranslate","id":"prcIsum"})
    
    if divs3 == None:
        df.loc[i,"Item Price"] = None
    else:
        item_price = re.findall("\$[.0-9].+", divs3.text)
        df.loc[i,"Item Price"]=item_price[0]
        print(df.loc[i,"Item Price"])
       
    

    #List Price
   
    div8 = soup.find("span",{"id":"orgPrc"})
    
    if div8 == None:
        df.loc[i,"List Price"] = None
    else:
        list_price = re.findall("\$[.0-9].+", div8.text)
        df.loc[i,"List Price"]=list_price[0]

   
    
   #Number of Items to be Sold
    div4 = soup.find("a",{"class":"vi-txt-underline"})
   
    if div4:
        try:
            item_sold = re.findall("[0-9,].+? ", div4.text)
            df.loc[i,"Number of Items Sold"] = item_sold[0][:-1]
        except:
            df.loc[i,"Number of Items Sold"] = None
        
        
    
    #Returns Allowed
    divs5 = soup.find_all("div",{"id":"returnsPlacementHolderId"})
    for div in divs5:
        if div.text != "":
            df.loc[i,"Returns Allowed"] = True
        else:
            df.loc[i,"Returns Allowed"] = False
            
    #Shipping Cost
    if soup.find("span",{"id":"shSummary"}) != None:
        div6 = soup.find("span",{"id":"shSummary"})
    else:
        div6 = soup.find("span",{"class":"vi-fnf-ship-txt"})
    
    if div6:
        if "FREE" in div6.text:
            df.loc[i,"Shipping Price"] = 0
          
        else:
            ship = re.findall("\$[0-9].+?", div6.text)
            df.loc[i,"Shipping Price"] = ship[0]
          
    else:
        df.loc[i,"Shipping Price"] = None
    
    #Condition
    div7 = soup.find("div",{"id":"vi-itm-cond"})
    df.loc[i,"Condition"] = div7.text
    
            
    
    f.close
 


$9.99
$20.99


In [139]:
df1=df.copy()
df1.head(4)

,Seller Name,Seller Score,Item Price,List Price,Number of Items Sold,Returns Allowed,Shipping Price,Condition
0,cozyarray,99.3,$9.99,$87.99,"181,367",True,0,New with tags
1,cozyarray,99.3,$20.99,$89.99,"22,166",True,0,New with tags
2,cozyarray,99.3,$13.99,$89.99,"21,122",True,0,New with tags
3,cozyarray,99.3,$5.99,$29.99,"27,859",True,0,New with tags


In [140]:
df2=df1.copy()
df2.head()

,Seller Name,Seller Score,Item Price,List Price,Number of Items Sold,Returns Allowed,Shipping Price,Condition
0,cozyarray,99.3,$9.99,$87.99,"181,367",True,0,New with tags
1,cozyarray,99.3,$20.99,$89.99,"22,166",True,0,New with tags
2,cozyarray,99.3,$13.99,$89.99,"21,122",True,0,New with tags
3,cozyarray,99.3,$5.99,$29.99,"27,859",True,0,New with tags
4,utopiadeals,99.6,$25.99/ea,None,"5,847",True,0,New with tags


In [141]:
df2["Item Price"]=(df1["Item Price"].replace(['\$','/ea'], ['',''], regex=True)).astype(float)
df2["List Price"]=(df1["List Price"].replace(['\$'], [''], regex=True)).astype(float)
df2["Number of Items Sold"]=df1["Number of Items Sold"].replace([','], [''], regex=True).astype(float)

In [142]:
#Converting all prices to dollar cent format

df2["Item Price"]=df2["Item Price"]*100
df2["List Price"]=df2["List Price"]*100
df2["Shipping Price"]=(df2["Shipping Price"].astype(float))*100

In [144]:
df2.replace(to_replace=[None], value=['NULL'], inplace=True)

In [145]:
df2["Returns Allowed"] = np.where(df['Returns Allowed']== True, 'True', 'False')

# Step 5

Connect to SQL (either MySQL, MariaDB, or SQLite. Do NOT use SQL GUI or command terminal). Save the information of items into a single table named "deals"in “msba”. If an item misses ANY of the information in (d), you should insert that missing value as NULL into the table. 

In [146]:
#Checking the db - msba

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Shreya1912@",
  database="msba"
)
mycursor = mydb.cursor()
mycursor = mydb.cursor(buffered=True)

In [147]:
mycursor.execute("Drop Table deals")

In [148]:
#Creating table

mycursor.execute("CREATE TABLE IF NOT EXISTS deals (Seller_Name VARCHAR(255), Seller_Score VARCHAR(255), Item_price INT default NULL, List_Price INT default NULL, Number_of_Items_Sold INT, Returns_Allowed VARCHAR(255), Shipping_Price INT, Condition_Item VARCHAR(255))")


In [149]:
#Inserting values into the table

for i in range(0,len(df2)):
    try:
        if(df2.iloc[i,3] != 'NULL'):
            if(df2.iloc[i,2] != 'NULL'):
                if(df2.iloc[i,4] != 'NULL'):
                    mycursor.execute("INSERT INTO deals (Seller_Name, Seller_Score, Item_Price, List_Price, Number_of_Items_Sold, Returns_Allowed, Shipping_Price, Condition_Item) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", (df2.iloc[i,0], df2.iloc[i,1], df2.iloc[i,2], df2.iloc[i,3], df2.iloc[i,4], df2.iloc[i,5], df2.iloc[i,6], df2.iloc[i,7]))
                    mydb.commit()
                else:
                    mycursor.execute("INSERT INTO deals (Seller_Name, Seller_Score, Item_Price, List_Price, Returns_Allowed, Shipping_Price, Condition_Item) VALUES (%s, %s, %s, %s, %s, %s, %s)", (df2.iloc[i,0], df2.iloc[i,1], df2.iloc[i,2], df2.iloc[i,3], df2.iloc[i,5], df2.iloc[i,6], df2.iloc[i,7]))
                    mydb.commit()
            else:
                if(df2.iloc[i,4] != 'NULL'):
                    mycursor.execute("INSERT INTO deals (Seller_Name, Seller_Score, List_Price, Number_of_Items_Sold, Returns_Allowed, Shipping_Price, Condition_Item) VALUES (%s, %s, %s, %s, %s,%s, %s)", (df2.iloc[i,0], df2.iloc[i,1], df2.iloc[i,3], df2.iloc[i,4], df2.iloc[i,5], df2.iloc[i,6], df2.iloc[i,7]))
                    mydb.commit()
                else:
                    mycursor.execute("INSERT INTO deals (Seller_Name, Seller_Score, List_Price,Returns_Allowed, Shipping_Price,Condition_Item) VALUES (%s, %s, %s, %s, %s, %s)", (df2.iloc[i,0], df2.iloc[i,1], df2.iloc[i,3], df2.iloc[i,5], df2.iloc[i,6], df2.iloc[i,7]))
                    mydb.commit()
        else:
            if(df2.iloc[i,2] != 'NULL'):
                if(df2.iloc[i,4] != 'NULL'):
                    mycursor.execute("INSERT INTO deals (Seller_Name, Seller_Score, Item_Price, Number_of_Items_Sold, Returns_Allowed, Shipping_Price, Condition_Item) VALUES (%s, %s, %s, %s, %s,%s, %s)", (df2.iloc[i,0], df2.iloc[i,1], df2.iloc[i,2], df2.iloc[i,4], df2.iloc[i,5], df2.iloc[i,6], df2.iloc[i,7]))
                    mydb.commit()
                else:
                    mycursor.execute("INSERT INTO deals (Seller_Name, Seller_Score, Item_Price, Returns_Allowed, Shipping_Price, Condition_Item) VALUES (%s, %s, %s, %s, %s, %s)", (df2.iloc[i,0], df2.iloc[i,1], df2.iloc[i,2], df2.iloc[i,5], df2.iloc[i,6], df2.iloc[i,7]))
                    mydb.commit()
            else:
                if(df2.iloc[i,4] != 'NULL'):
                    mycursor.execute("INSERT INTO deals (Seller_Name, Seller_Score, Number_of_Items_Sold, Returns_Allowed, Shipping_Price,Condition_Item) VALUES (%s, %s, %s, %s, %s, %s)", (df2.iloc[i,0], df2.iloc[i,1], df2.iloc[i,4], df2.iloc[i,5], df2.iloc[i,6], df2.iloc[i,7]))
                    mydb.commit()
                else:
                    mycursor.execute("INSERT INTO deals (Seller_Name, Seller_Score, Returns_Allowed, Shipping_Price, Condition_Item) VALUES (%s, %s, %s, %s, %s)", (df2.iloc[i,0], df2.iloc[i,1], df2.iloc[i,5], df2.iloc[i,6], df2.iloc[i,7]))
                    mydb.commit()
    except:
        pass


In [150]:
mycursor.execute("SELECT * from deals")
myresult = mycursor.fetchall()

for x in myresult:
    print(x)

('cozyarray', '99.3', 999, 8799, 181367, 'True', 0, 'New with tags')
('cozyarray', '99.3', 2099, 8999, 22166, 'True', 0, 'New with tags')
('cozyarray', '99.3', 1399, 8999, 21122, 'True', 0, 'New with tags')
('cozyarray', '99.3', 599, 2999, 27859, 'True', 0, 'New with tags')
('utopiadeals', '99.6', 2599, None, 5847, 'True', 0, 'New with tags')
('cozyarray', '99.3', 899, 3999, 8311, 'True', 0, 'New with tags')
('utopiadeals', '99.6', 1500, None, 3531, 'True', 0, 'New with tags')
('cozyarray', '99.3', 1499, 9999, 6791, 'True', 0, 'New with tags')
('homecollection_deals', '99.1', 2250, None, 4256, 'True', 0, 'New with tags')
('cozyarray', '99.3', 699, 6999, 21667, 'True', 0, 'New with tags')
('cozyarray', '99.3', 1099, 2499, 4630, 'True', 0, 'New with tags')
('cozyarray', '99.3', 1399, 6999, 5738, 'True', 0, 'New with tags')
('cozyarray', '99.3', 2199, 10198, 80826, 'True', 0, 'New with tags')
('utopiadeals', '99.6', 1799, None, 2491, 'True', 0, 'New with tags')
('luxdecorcollection', '99.

# Step 6

Use your code script to run summary stats on each item. Print to the screen the mean, min, max, and mean for each numeric column, dependent on (i) whether “list price”was providedand (ii) "condition" (group by at the same time, not separately).

In [ ]:
#Using SQL query

In [151]:
#Summary statistics when list price is not null

mycursor.execute("Select Condition_Item, min(Item_price), max(Item_price), avg(Item_price), min(List_price), max(List_price), avg(List_price),min(Number_of_Items_Sold), max(Number_of_Items_Sold), avg(Number_of_Items_Sold) from msba.deals where List_price is not Null group by Condition_Item")
myresult = mycursor.fetchall()

for x in myresult:
    print(x)

('New with tags', 599, 7982, Decimal('2485.4691'), 1699, 24999, Decimal('7948.8642'), 12, 181367, Decimal('8433.6250'))
('New', 999, 9499, Decimal('2075.0000'), 2339, 40595, Decimal('5681.6316'), 72, 28130, Decimal('2473.3889'))


In [152]:
#Summary statistics when list price is null

mycursor.execute("Select Condition_Item, min(Item_price), max(Item_price), avg(Item_price), min(Number_of_Items_Sold), max(Number_of_Items_Sold), avg(Number_of_Items_Sold) from msba.deals where List_price is Null group by Condition_Item")
myresult = mycursor.fetchall()

for x in myresult:
    print(x)

('New with tags', 599, 13499, Decimal('2754.4318'), 24, 6976, Decimal('1506.4217'))
('New', 550, 3999, Decimal('1566.0000'), 59, 6262, Decimal('1973.1250'))


In [ ]:
#Using python summary stats function

In [153]:
df3 = df2.copy()

In [156]:
df3.replace(to_replace=['NULL'], value=[None], inplace=True)

In [157]:
df3['Seller Score'] = df3['Seller Score'].astype(float)
df3['Item Price'] = df3['Item Price'].astype(float)
df3['List Price'] = df3['List Price'].astype(float)
df3['Number of Items Sold'] = df3['Number of Items Sold'].astype(float)
df3['Shipping Price'] = df3['Shipping Price'].astype(float)
#df3["List Price Status"] = np.where(df3['List Price']!= None, 'Yes', 'No')

In [158]:
#Making df with items having list price
df_list = df3[df3['List Price'].notnull()]

In [159]:
#Making df with items having no list price
df_no_list = df3[df3['List Price'].isnull()]

In [160]:
df_list.groupby('Condition').agg({'Seller Score': ['min', 'max', 'mean'],       
        'Item Price': ['min', 'max', 'mean'],       
        'List Price': ['min', 'max', 'mean'],       
        'Number of Items Sold': ['min', 'max', 'mean'],       
        'Shipping Price': ['min', 'max', 'mean']})

Seller Score                   Item Price                       \
                       min    max       mean        min     max         mean   
Condition                                                                      
New                   99.3  100.0  99.852632      999.0  9499.0  2075.000000   
New with tags         98.9  100.0  99.358025      599.0  7982.0  2485.469136   

              List Price                       Number of Items Sold            \
                     min      max         mean                  min       max   
Condition                                                                       
New               2339.0  40595.0  5681.631579                 72.0   28130.0   
New with tags     1699.0  24999.0  7948.864198                 12.0  181367.0   

                           Shipping Price            
                      mean            min  max mean  
Condition                                            
New            2473.388889            0.0  0.0  0.0  
New with tags  8433.625000            0.0  0.0  0.0

In [161]:
df_no_list.groupby('Condition').agg({'Seller Score': ['min', 'max', 'mean'],       
        'Item Price': ['min', 'max', 'mean'],       
        'List Price': ['min', 'max', 'mean'],       
        'Number of Items Sold': ['min', 'max', 'mean'],       
        'Shipping Price': ['min', 'max', 'mean']})

Seller Score                   Item Price                        \
                       min    max       mean        min      max         mean   
Condition                                                                       
New                   99.1  100.0  99.287500      550.0   3999.0  1566.000000   
New with tags         98.0  100.0  99.171579      599.0  13499.0  2754.431818   

              List Price          Number of Items Sold                       \
                     min max mean                  min     max         mean   
Condition                                                                     
New                  NaN NaN  NaN                 59.0  6262.0  1973.125000   
New with tags        NaN NaN  NaN                 24.0  6976.0  1506.421687   

              Shipping Price            
                         min  max mean  
Condition                               
New                      0.0  0.0  0.0  
New with tags            0.0  0.0  0.0

# Insights

In [ ]:
#We can see from the table 1, that when list price is mentioned, mean number of items sold is large compared 
#to when the list price is not printed. This is true for both item condition types - new and new with tags
#Infact, the mean item price is higher for item condition new when the list price is mentioned compared to cases when 
#it is not. 
#For items with contition - new with tags, Item price for items when list price is mentioned is slighly lower than that of when the list price is not mentioned 
#But the number of items in the former case is much higher than the latter case
#With these observations, overall we can say that mentioning list prices drives up sales.